Group Name: AG xx.

Student Name (Student ID):

1. Wong Ji Fong (A0249572U)

2. xxxx xxxxx (xxxxxxx)

3. xxxx xxxxx (xxxxxxx)

In [1]:
# Type hinting is used within this document to help understand parameters
from __future__ import annotations
from typing import Tuple, List, Optional, Set, Union

# Question 1

Consider the maze shown below. The Maze has 16 rows and 24 columns The objective is to find a shortest path from cell $S$ to cell $G$.


![Maze](Maze_Assignment_1-1.jpg)


The agent can take four actions in each cell: 'RIGHT', 'DOWN', 'UP', 'LEFT'.  

Each cell is represented as $(x,y)$, where $x$ indicates row number and $y$ indicates column number. Action 'UP' takes the agent from cell $(x,y)$ to $(x+1,y)$. Action 'DOWN' takes the agent from cell $(x,y)$ to $(x-1,y)$. Action 'RIGHT' takes the agent from cell $(x,y)$ to $(x,y+1)$. Action 'LEFT' takes the agent from cell $(x,y)$ to $(x,y-1)$. The triplet $(s,a,s')$  indicates that taking action $a$ at state $s$ leads to state $s'$. Actions 'LEFT' or 'RIGHT' cost 10 units for all $(s,a,s')$. Actions 'UP' or 'DOWN' cost 1 unit for all  $(s,a,s')$.  The agent cannot move into cells that are shaded. Assume that the agent knows the boundaries of the maze and has full observability. Consequently, at the bottom (row 0) and top (row 15), the agent will not take actions 'DOWN' and 'UP', respectively; at left (column 0) and right (column 23) columns, the agent will not take 'LEFT' and 'RIGHT' actions, respectively. Similalry, the agent will not take actions that lead to shaded region in the maze.

## **Q1.a: Class Maze(Problem)** [3 Marks]

Write a Maze class to create a model for this problem. You should not use an explicit state space model. The modelling should inherit the abstract class 'Problem' (given below). With the problem formulation, find the shortest path from S to G cell. Propose and implement multiple heuristics (at least two heuristics) for informed search algorithms. 

## **Q1.b: Analysis of the Algorithms** [7 Marks]

1. Solve the above Maze problem using the following algorithms

    a. Breadth-First Search

    b. Depth-First Search with Cycle-Check

    c. Iterative-Deepening Search with Cycle-Check

    d. Uniform-Cost Search

    e. A* Search 

    f. Greedy Best-first Search

    g. Any other variants for search algorithms that are not discussed in the class (bonus/optional question) 

2. Identify the number of nodes generated, number of nodes expanded, maximum frontier size, and path-cost for the above algorithms. 
 
3. Compare the performance of informed search algorithms with proposed heuristics. Identify the best performing heuristic and explain.
 
4. Draw a bar plot comparing the statistics of the algorithms and explain the results. 

Note 1: You must follow the problem formulation discussed in the class. A abstract class for Problem amd Node definition is presented below. The search tree generation should follow the template discussed in the class (i.e., Node class, expand methods, etc.). 

Note 2: If you are borrowing a block of code (for example, helper functions or data structures, etc.) from AIMA4e repository, you have to acknowledge it in the code. 

Note 3: The code should be written in a single jupyter notebook file.

In [2]:
class Problem:
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When you create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)

In [3]:
# Use the following Node class to generate search tree
import math

class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost 
    
    # newly defined to assist with trace
    def __eq__(self, other: Node): return self.state == other.state
    def set_parent(self, other: Node, action_cost: int):
        """
        Args:
            other (Node): The other node which to assign as parent node
            action_cost (int): Cost of the action taken to take this node
        """
        self.parent = other
        self.path_cost = other.path_cost + action_cost
        return self

In [4]:
class Maze(Problem):
    def __init__(self,
                 initial: Node,
                 goal: Node,
                 boundaries: Tuple[int, int],
                 action_cost_map: Optional[dict],
                 visited: Optional[List[Node]]=None,
                 **kwds):
        """
        Add type hints and parameter to know boundaries
        given the assumption "Assume that the agent knows
        the boundaries of the maze and has full observability"
        
        Args:
            initial (Node): Node for the initial state
            goal (Node): Node for the goal state
            boundaries (Tuple[int,int]):
                (rows, cols) of boundaries, assuming (0,0) to (rows,cols) 
                as problem space
            visited (Optional[List[Node]]):
                If the search algorithm require a visited list / priority queue
                this should be provided
        """
        super().__init__(initial=initial,
                         goal=goal,boundaries=boundaries,
                         action_cost_map=action_cost_map,
                         **kwds)
    
    def action_cost(self, s: Node, a: str, s1: Node) -> int:
        """
        Args:
            s (Node): Node from current state
            a (str): Action to take
            s1 (Node): Node to travel to
        
        Returns:
            int: Cost (s, a, s')
        """
        return self.action_cost_map[a]
    
    def _transform_permissable_action(self,
                                      actions: Tuple[Tuple[int, int], str],
                                     current_node: Node) -> Tuple[int, Node, str]:
        state, action = actions
        new_node = Node(state)
        
        action_cost = self.action_cost(state, action, new_node)
        new_node.set_parent(current_node, action_cost)
        return (action_cost, new_node, action)
        
    def actions(self, state: Node) -> List[Tuple[str, int]]:
        """
        Return permissable actions as list of actions
        as (COST, s', ACTION)
        Args:
            state (Node): Agents current state node
        
        Returns:
            List[Tuple[int, Node, str]]:
                (COST (int), s' (Node), ACTION (str))
        """
        x = state.state[0]
        y = state.state[1]
        parent = state.parent if state.parent else None
        
        permissable_x_y = {((x, y+1), 'UP'),
                           ((x, y-1), 'DOWN'),
                           ((x+1, y), 'RIGHT'),
                           ((x-1,y), 'LEFT'),}
        
        return list(
                map(
                        lambda actions: self._transform_permissable_action(actions, state),
                        filter(
                            lambda actions:
                                   # Ensuring they permissable actions are:
                                   # * within boundaries
                                   # * not parent, previous node
                                   # * not a shaded region
                                   0<=actions[0][0]<=self.boundaries[0] and
                                   0<=actions[0][1]<=self.boundaries[1] and
                                   (actions[0][0], actions[0][1]) != parent and
                                   (actions[0][0], actions[0][1]) not in self.shaded_regions,
                               permissable_x_y
                        )
                )
        )
    
    def h(self, node: Node) -> Union[float, int]:
        """
        Implementing Manhatten distance
        Args:
            node (Node): Current agent node
        
        Returns:
            Union[float, int]: heuristic value
        """
        return abs(self.goal.state[0]-node.state[0]) + abs(self.goal.state[1]-node.state[1])
    
    def h2(self, node: Node) -> Union[float, int]:
        """
        Implementing Euclidean distance
        Args:
            node (Node): Current agent node
        
        Returns:
            Union[float, int]: heuristic value
        """
        return ((self.goal.state[0]-node.state[0])**2 + 
                (self.goal.state[1]-node.state[1])**2)**(1/2)
        
        
    def __repr__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)

### Instantiating start and goal nodes

In [5]:
start_node = Node((10,8))
goal_node = Node((9,11))

### Instantiating maze with start, end, maze size, boundaries, action costs and shaded regions (not known to agent itself)

In [6]:
maze = Maze(start_node,
            goal_node,
            boundaries=(23, 15),
            action_cost_map={'LEFT':10, 'RIGHT': 10, 'UP': 1, 'DOWN': 1},
            shaded_regions = {(9,6),
                               (9,7),
                               (9,8),
                               (9,9),
                               (9,10),
                               (10,10),
                               (10,11),
                               (10,12),
                               (10,13),
                               (9,14),
                               (12,10),
                               (13,10),
                               (12,11),
                               (13,11)})

### Example path to show permissable actions taking into account restrictions

In [7]:
actions = maze.actions(Node((0,0)))

In [8]:
# List of permissable actions. From 0,0, agent can only move up and right.
actions

[(1, <(0, 1)>, 'UP'), (10, <(1, 0)>, 'RIGHT')]

In [9]:
# Taking the right,
node_1 = actions[1][1]
actions2 = maze.actions(node_1)
node_1

<(1, 0)>

In [10]:
actions2

[(10, <(2, 0)>, 'RIGHT'), (10, <(0, 0)>, 'LEFT'), (1, <(1, 1)>, 'UP')]

In [11]:
# Taking the right would yield the total path cost so far or 20, 10 for each right made
node_2 = actions2[0][1]
node_2.path_cost

20

In [12]:
# Example of where agent will only be able to choose
# up/down due to shaded areas
maze.actions(Node((11,10)))

[(1, <(11, 11)>, 'UP'), (1, <(11, 9)>, 'DOWN')]

### Per the comparison dunder method defined, comparing the nodes so far yields that node_2 has a higher pathcost than node_1

In [13]:
node_2 < node_1

False

### Heuristics 1 and 2
Defining an arbitrary node

In [14]:
intermediate_node = Node((12,9))

In [15]:
print(f"Manhatten distance: {maze.h(intermediate_node)}")
print(f"Euclidean distance: {maze.h2(intermediate_node)}")

Manhatten distance: 5
Euclidean distance: 3.605551275463989


In [16]:
# Tests, please ignore but use for testing classes
print(start_node == Node((10, 8)))
print(goal_node == Node((9, 11)))

True
True
